# Notebook 5: Macroeconomic and Market Data Download, Cleaning, and Preparation  
# Author: Will Wu

---

## **Table of Contents** ##

1. [Macro Data Download](#1)
2. [Feature Engineering](#2)
3. [Federal Funds Rate](#3)
4. [Timing and Data Alignment Considerations](#4)
5. [Mapping Macroeconomic Data to Daily Frequency](#5)
    - [CPI](#5a)
    - [PPI](#5b)
    - [PCE](#5c)
    - [Unemployment Rate](#5d)
    - [GDP](#5e)
    - [Federal Funds Rate](#5f)
6. [Aggregate the Macro Data](#6)



---

## Notebook Introduction

This notebook focuses on downloading macroeconomic and market data from online resources. It proceeds with data cleaning, feature engineering, and mapping the macro data onto daily yield change data. Since most macroeconomic indicators are released less frequently than daily, careful mapping is essential to avoid look-ahead bias and accurately reflect the timing of data availability in the real world. This approach ensures that the macro data's influence on daily yield changes is represented realistically and chronologically correctly.

---


## 1. Macro Data Download <a class="anchor" id="1"></a>

In [371]:
# Install the required packages to download data from FRED api
!pip install fredapi

In [372]:
# Install the required package to download Federal Funds Rate 
!pip install pandas-datareader

In [373]:
# Import the necessary packages for this notebook
from fredapi import Fred
import pandas_datareader.data as web
import pandas as pd
from datetime import datetime

- Load dataset

In [374]:
# Import the yield changes dataset
yield_changes_df = pd.read_csv("../data/yield_changes_from_3_month.csv", index_col=0)

# Convert index to datetime
yield_changes_df.index = pd.to_datetime(yield_changes_df.index)

# Check head()
yield_changes_df.head(3)

,3_month,6_month,1_year,2_year,3_year,5_year,7_year,10_year,20_year,30_year
2019-01-03,-0.01,-0.04,-0.10,-0.11,-0.12,-0.12,-0.12,-0.10,-0.08,-0.05
2019-01-04,0.01,0.04,0.07,0.11,0.12,0.12,0.12,0.11,0.08,0.06
2019-01-07,0.03,0.03,0.01,0.03,0.04,0.04,0.04,0.03,0.03,0.01


In [375]:
# Import the yield dataset
yield_curve = pd.read_csv("../data/yield_cleaned.csv", index_col=0)

# Convert index to datetime
yield_curve.index = pd.to_datetime(yield_curve.index)

# To match the yield changes dataset, we need to remove the first line and 1_month column
yield_df = yield_curve.iloc[1:, 1:].copy()

# Check head
yield_df.head(3)

,3_month,6_month,1_year,2_year,3_year,5_year,7_year,10_year,20_year,30_year
2019-01-03,2.41,2.47,2.50,2.39,2.35,2.37,2.44,2.56,2.75,2.92
2019-01-04,2.42,2.51,2.57,2.50,2.47,2.49,2.56,2.67,2.83,2.98
2019-01-07,2.45,2.54,2.58,2.53,2.51,2.53,2.60,2.70,2.86,2.99


- Download data from FRED API

In [376]:
# Input my FRED API key
fred = Fred(api_key='2ac8fe087628e3a924f92c841a2fad5a')

# Specify start and end dates, add 2018 data for calculating YoY growth
start_date = '2017-09-01'
end_date = '2025-10-22'

# 1. CPI (Consumer Price Index)
cpi = fred.get_series('CPIAUCSL', observation_start=start_date, observation_end=end_date)

# 2. PPI (Producer Price Index)
ppi = fred.get_series('PPIACO', observation_start=start_date, observation_end=end_date)

# 3. PCE (Personal Consumption Expenditures Price Index) (core PCE)
pce = fred.get_series('PCEPI', observation_start=start_date, observation_end=end_date)

# 4. Unemployment Rate
u_rate = fred.get_series('UNRATE', observation_start=start_date, observation_end=end_date)

# 5. US GDP (Quarterly, interpolate to daily if needed)
gdp = fred.get_series('GDP', observation_start=start_date, observation_end=end_date)

# 6. Federal Funds Rate (Policy rate)
ff_rate = fred.get_series('FEDFUNDS', observation_start=start_date, observation_end=end_date)


# Convert to DataFrames
cpi_df = cpi.to_frame(name='CPI')
ppi_df = ppi.to_frame(name='PPI')
pce_df = pce.to_frame(name='PCE')
u_rate_df = u_rate.to_frame(name='Unemployment Rate')
gdp_df = gdp.to_frame(name='GDP')
ff_rate_df = ff_rate.to_frame(name='Fed Funds Rate')

In [377]:
# Check the head of each data
dataframe_list = [cpi_df, ppi_df, pce_df, u_rate_df, gdp_df, ff_rate_df]

for df in dataframe_list:
    display(df.head(3))

,CPI
2017-09-01,246.435
2017-10-01,246.626
2017-11-01,247.284


,PPI
2017-09-01,194.8
2017-10-01,194.9
2017-11-01,195.9


,PCE
2017-09-01,100.400
2017-10-01,100.531
2017-11-01,100.675


,Unemployment Rate
2017-09-01,4.3
2017-10-01,4.2
2017-11-01,4.2


,GDP
2017-07-01,19692.595
2017-10-01,20037.088
2018-01-01,20328.553


,Fed Funds Rate
2017-09-01,1.15
2017-10-01,1.15
2017-11-01,1.16


#### Insights and Goals:

CPI, PPI, and PCE are provided monthly, and GDP is reported quarterly—consistent with their release frequencies. The Fed Funds Rate is typically updated every six weeks; within our dataset, it appears to be updated roughly once a month. This discrepancy suggests that further investigation is needed to fully understand the data’s update schedule and potential gaps.

After an initial review of the macroeconomic data, the following preprocessing steps are necessary before integrating these variables into our daily yield change dataset:
1. Identify whether the data pertains to the previous or current period (month/quarter) to prevent look-ahead bias. For example, we should avoid using March CPI data (released in mid-April) to forecast the March yield curve.
2. Determine the actual release dates of the macroeconomic data (e.g., CPI, PPI, PCE), since they are often announced mid-month but recorded on the first day of the month. Accurate timing is crucial for precise prediction.
3. Feature engineering macroeconmic data such as adding YoY change in CPI, etc. since YoY change in CPI is usually more important for understanding the inflation.
3. Map these aggregate data onto daily frequency by appropriately duplicating or interpolating values. This ensures the macro variables align correctly with daily yield change data for accurate forecasting without data leakage.

---

## 2 Feature engineering <a class="anchor" id="2"></a>

To enhance our analysis and better capture the dynamics of key economic indicators, we will generate new features representing their recent trends. Specifically, we will create columns for **Year-over-Year (YoY)** and **Quarter-over-Quarter (QoQ)** percentage changes for CPI, PPI, PCE, unemployment rate, GDP. These features will allow us to analyze both long-term trends and short-term momentum, facilitating more informative modeling and insights.

In [378]:
# CPI
# Add YoY change
cpi_df['CPI_yoy_change'] = (cpi_df['CPI'].diff(12) / cpi_df['CPI'].shift(12)) * 100

# drop NaN in yoy_change column
cpi_df = cpi_df.dropna()

# Check the head
cpi_df.head()

,CPI,CPI_yoy_change
2018-09-01,252.182,2.332055
2018-10-01,252.772,2.492032
2018-11-01,252.594,2.147329
2018-12-01,252.767,2.002381
2019-01-01,252.561,1.487589


In [379]:
# PPI
# Add YoY change
ppi_df['PPI_yoy_change'] = (ppi_df['PPI'].diff(12) / ppi_df['PPI'].shift(12)) * 100

# drop NaN in yoy_change column
ppi_df = ppi_df.dropna()

# Check the head
ppi_df.head()

,PPI,PPI_yoy_change
2018-09-01,203.6,4.517454
2018-10-01,204.6,4.976911
2018-11-01,202.3,3.266973
2018-12-01,201.0,2.394294
2019-01-01,199.1,0.606367


In [380]:
# PCE
# Add YoY change
pce_df['PCE_yoy_change'] = (pce_df['PCE'].diff(12) / pce_df['PCE'].shift(12)) * 100

# drop NaN in yoy_change column
pce_df = pce_df.dropna()

# Check the head
pce_df.head()

,PCE,PCE_yoy_change
2018-09-01,102.418,2.009960
2018-10-01,102.595,2.053098
2018-11-01,102.630,1.941892
2018-12-01,102.701,1.864691
2019-01-01,102.648,1.431832


In [381]:
# Unemployment rate
# Add YoY change and MoM change
u_rate_df['Unemployment_mom_change'] = (u_rate_df['Unemployment Rate'].diff() / u_rate_df['Unemployment Rate'].shift(1)) * 100
u_rate_df['Unemployment_yoy_change'] = (u_rate_df['Unemployment Rate'].diff(12) / u_rate_df['Unemployment Rate'].shift(12)) * 100

# drop NaN in yoy_change column
u_rate_df = u_rate_df.dropna()

# Check the head
u_rate_df.head()

,Unemployment Rate,Unemployment_mom_change,Unemployment_yoy_change
2018-09-01,3.7,-2.631579,-13.953488
2018-10-01,3.8,2.702703,-9.523810
2018-11-01,3.8,0.000000,-9.523810
2018-12-01,3.9,2.631579,-4.878049
2019-01-01,4.0,2.564103,0.000000


In [382]:
# Unemployment rate
# Add YoY change and QoQ change
gdp_df['GDP_qoq_change'] = (gdp_df['GDP'].diff(1) / gdp_df['GDP'].shift(1)) * 100
gdp_df['GDP_yoy_change'] = (gdp_df['GDP'].diff(4) / gdp_df['GDP'].shift(4)) * 100

# drop NaN in yoy_change column
gdp_df = gdp_df.dropna()

# Check the head
gdp_df.head()

,GDP,GDP_qoq_change,GDP_yoy_change
2018-07-01,20798.730,1.058350,5.617010
2018-10-01,20917.867,0.572809,4.395744
2019-01-01,21111.600,0.926160,3.851956
2019-04-01,21397.938,1.356306,3.969824
2019-07-01,21717.171,1.491887,4.415851


---

## 3 Federal Funds Rate <a class="anchor" id="3"></a>

In [383]:
# Check the current Federal Fund Rate Data
ff_rate_df.head(6)

,Fed Funds Rate
2017-09-01,1.15
2017-10-01,1.15
2017-11-01,1.16
2017-12-01,1.30
2018-01-01,1.41
2018-02-01,1.42



After an inspection of Federal Funds Rate: I found that the data from FRED for the Federal Funds Rate reflects the effective rate, which is an average influenced by the Federal Reserve's monetary policy. In finance, the target federal funds rate is set by the FOMC as the benchmark interest rate for overnight interbank loans. It is typically announced during FOMC meetings held roughly every six weeks. Since this target rate impacts the yield curve more directly, I should update our data source to use the target funds rate for better accuracy in our analysis.

In [384]:
# Define the FRED series IDs for the Federal Funds Target Range
series_ids = ['DFEDTARL', 'DFEDTARU']
start_date = '2017-09-01'
end_date = '2025-10-22'

try:
    # Fetch the data for the Target Rate Lower (L) and Upper (U) bounds
    target_rate_data = web.DataReader(series_ids, 'fred', start=start_date, end=end_date)
    
    # Rename columns for clarity
    target_rate_data.columns = ['Fed_Fund_Rate_Lower', 'Fed_Fund_Rate_Lower_Upper']
    
    # The rate only changes on announcement dates
    # We must use .dropna(how='all') because FRED data might have some missing days
    target_rate_changes = target_rate_data.dropna(how='all').drop_duplicates()
    
    print("--- FOMC Federal Funds Target Rate Changes (2018-01-01 to 2025-10-22) ---")
    print(target_rate_changes)
    
except Exception as e:
    print(f"An error occurred: {e}")

--- FOMC Federal Funds Target Rate Changes (2018-01-01 to 2025-10-22) ---
            Fed_Fund_Rate_Lower  Fed_Fund_Rate_Lower_Upper
DATE                                                      
2017-09-01                 1.00                       1.25
2017-12-14                 1.25                       1.50
2018-03-22                 1.50                       1.75
2018-06-14                 1.75                       2.00
2018-09-27                 2.00                       2.25
2018-12-20                 2.25                       2.50
2020-03-16                 0.00                       0.25
2022-03-17                 0.25                       0.50
2022-05-05                 0.75                       1.00
2022-09-22                 3.00                       3.25
2022-11-03                 3.75                       4.00
2022-12-15                 4.25                       4.50
2023-02-02                 4.50                       4.75
2023-03-23                 4.75          

In [385]:
# Feature Engineer
# Get the rate change between current and last announcement
fed_rate_df = target_rate_changes.copy()
fed_rate_df['rate_change_bps'] = fed_rate_df['Fed_Fund_Rate_Lower'].diff() * 100

# Only keep the data after 2019, which is announced at 2018-12-20
fed_rate_df = fed_rate_df.loc['2018-12-20':, :]

# Show the head
fed_rate_df

,Fed_Fund_Rate_Lower,Fed_Fund_Rate_Lower_Upper,rate_change_bps
DATE,,,
2018-12-20,2.25,2.50,25.0
2020-03-16,0.00,0.25,-225.0
2022-03-17,0.25,0.50,25.0
2022-05-05,0.75,1.00,50.0
2022-09-22,3.00,3.25,225.0
2022-11-03,3.75,4.00,75.0
2022-12-15,4.25,4.50,50.0
2023-02-02,4.50,4.75,25.0
2023-03-23,4.75,5.00,25.0


Now the Federal Funds Data is the target rate discussed in each FOMC meeting, and we have added a rate change column to represent the change in funds change magnitude in bps.

---

## 4 Timing and Data Alignment Considerations <a class="anchor" id="4"></a>

In this section, I will identify whether the data pertains to the previous or current period to prevent look-ahead bias. For example, we should avoid using March CPI data (released in mid-April) to forecast the March yield curve.

- CPI
<br>In 2021, CPI data was highly volatile, with the year-over-year CPI growth rate sharply increasing from 1.3% to over 7% by the end of the year. To analyze this, I will first calculate the year-over-year CPI change in our cpi_df. Subsequently, I will compare these results with the announced data dates. This comparison will assist in accurately mapping the monthly CPI data onto daily dates, ensuring that I avoid any look-ahead bias

In [386]:
# Show the CPI data from 2021
cpi_df.loc['2021-01-01': '2021-12-01', :]

,CPI,CPI_yoy_change
2021-01-01,262.639,1.355320
2021-02-01,263.573,1.667502
2021-03-01,264.847,2.623646
2021-04-01,266.625,4.137373
2021-05-01,268.404,4.926467
2021-06-01,270.710,5.317419
2021-07-01,271.965,5.269168
2021-08-01,272.752,5.181323
2021-09-01,273.942,5.363523
2021-10-01,276.528,6.226591


Here is the data I found online: 

| Reference Month  | CPI-U Index Value  | Release Date   | YoY Percent Change (Inflation Rate) |
|------------------|-------------------------|--------------------|------------------------------------|
| January 2021     | 261.582                 | February 10, 2021  | 1.4%                               |
| February 2021    | 263.014                 | March 10, 2021     | 1.7%                               |
| March 2021       | 264.877                 | April 13, 2021     | 2.6%                               |
| April 2021       | 267.054                 | May 12, 2021       | 4.2%                               |
| May 2021         | 269.195                 | June 10, 2021      | 5.0%                               |
| June 2021        | 271.696                 | July 13, 2021      | 5.4%                               |
| July 2021        | 273.003                 | August 11, 2021    | 5.4%                               |
| August 2021      | 273.567                 | September 14, 2021 | 5.3%                               |
| September 2021   | 274.310                 | October 13, 2021   | 5.4%                               |
| October 2021     | 276.589                 | November 10, 2021  | 6.2%                               |
| November 2021    | 277.948                 | December 10, 2021  | 6.8%                               |
| December 2021    | 278.802                 | January 12, 2022   | 7.0%                               |

### Analysis of CPI Data Timing and Mapping

After comparing the data, we observe that the CPI figures in our `cpi_df` actually correspond to the numbers published in the following month. For example, the YoY CPI change was 2.6% in March 2021, which is reflected in the row labeled '2021-03-01'. However, this data is not released until April 13th, 2021. 

Therefore, to accurately map the monthly CPI data onto the daily yield changes without introducing look-ahead bias, we need to use the CPI data **only after** the release date (e.g., after April 13th, 2021 for March's CPI). This ensures that any analysis respects the actual publication schedule and prevents future information from influencing earlier data points.

---

## 5 Mapping Macroeconomic Data to Daily Frequency <a class="anchor" id="5"></a>

In this section, for each macroeconomic variable, I will:  

1. Fetch the actual release dates of the data
2. Map each reference date in our dataset to the corresponding release date
3. Convert monthly or quarterly data into daily data while avoiding look-ahead bias
4. Filter the daily data to align with the trading days in our yield changes DataFrame

---

### 5.1 CPI <a class="anchor" id="5a"></a>

In [387]:
cpi_df.iloc[-1, :]

CPI               324.3680
CPI_yoy_change      3.0227
Name: 2025-09-01 00:00:00, dtype: float64

In [388]:
# ---  Define the Official CPI Release Date Mapping ---
# Key: The first day of the data's reference month (e.g., '2019-01-01' for Jan 2019 data)
# Value: The official BLS release date (the date the public sees the data)

CPI_RELEASE_MAP = {
    # --- 2017 ---
    # Key: Reference Month | Value: Release Date
    datetime(2017, 9, 1): datetime(2017, 10, 13),
    datetime(2017, 10, 1): datetime(2017, 11, 15),
    datetime(2017, 11, 1): datetime(2017, 12, 13),

    # --- 2018 ---
    datetime(2017, 12, 1): datetime(2018, 1, 12),
    datetime(2018, 1, 1): datetime(2018, 2, 14),
    datetime(2018, 2, 1): datetime(2018, 3, 13),
    datetime(2018, 3, 1): datetime(2018, 4, 11),
    datetime(2018, 4, 1): datetime(2018, 5, 10),
    datetime(2018, 5, 1): datetime(2018, 6, 12),
    datetime(2018, 6, 1): datetime(2018, 7, 12),
    datetime(2018, 7, 1): datetime(2018, 8, 10),
    datetime(2018, 8, 1): datetime(2018, 9, 13),
    datetime(2018, 9, 1): datetime(2018, 10, 11),
    datetime(2018, 10, 1): datetime(2018, 11, 14),
    datetime(2018, 11, 1): datetime(2018, 12, 11),

    # --- 2019 (Your original map starts here) ---
    datetime(2018, 12, 1): datetime(2019, 1, 11),
    datetime(2019, 1, 1): datetime(2019, 2, 13),
    datetime(2019, 2, 1): datetime(2019, 3, 12),
    datetime(2019, 3, 1): datetime(2019, 4, 16),
    datetime(2019, 4, 1): datetime(2019, 5, 10),
    datetime(2019, 5, 1): datetime(2019, 6, 12),
    datetime(2019, 6, 1): datetime(2019, 7, 16),
    datetime(2019, 7, 1): datetime(2019, 8, 13),
    datetime(2019, 8, 1): datetime(2019, 9, 12),
    datetime(2019, 9, 1): datetime(2019, 10, 10),
    datetime(2019, 10, 1): datetime(2019, 11, 13),
    datetime(2019, 11, 1): datetime(2019, 12, 11),

    # --- 2020 ---
    datetime(2019, 12, 1): datetime(2020, 1, 14),
    datetime(2020, 1, 1): datetime(2020, 2, 13),
    datetime(2020, 2, 1): datetime(2020, 3, 11),
    datetime(2020, 3, 1): datetime(2020, 4, 10),
    datetime(2020, 4, 1): datetime(2020, 5, 12),
    datetime(2020, 5, 1): datetime(2020, 6, 10),
    datetime(2020, 6, 1): datetime(2020, 7, 14),
    datetime(2020, 7, 1): datetime(2020, 8, 12),
    datetime(2020, 8, 1): datetime(2020, 9, 11),
    datetime(2020, 9, 1): datetime(2020, 10, 13),
    datetime(2020, 10, 1): datetime(2020, 11, 12),
    datetime(2020, 11, 1): datetime(2020, 12, 10),

    # --- 2021 ---
    datetime(2020, 12, 1): datetime(2021, 1, 13),
    datetime(2021, 1, 1): datetime(2021, 2, 10),
    datetime(2021, 2, 1): datetime(2021, 3, 10),
    datetime(2021, 3, 1): datetime(2021, 4, 13),
    datetime(2021, 4, 1): datetime(2021, 5, 12),
    datetime(2021, 5, 1): datetime(2021, 6, 10),
    datetime(2021, 6, 1): datetime(2021, 7, 13),
    datetime(2021, 7, 1): datetime(2021, 8, 11),
    datetime(2021, 8, 1): datetime(2021, 9, 14),
    datetime(2021, 9, 1): datetime(2021, 10, 13),
    datetime(2021, 10, 1): datetime(2021, 11, 10),
    datetime(2021, 11, 1): datetime(2021, 12, 10),

    # --- 2022 ---
    datetime(2021, 12, 1): datetime(2022, 1, 12),
    datetime(2022, 1, 1): datetime(2022, 2, 10),
    datetime(2022, 2, 1): datetime(2022, 3, 10),
    datetime(2022, 3, 1): datetime(2022, 4, 12),
    datetime(2022, 4, 1): datetime(2022, 5, 11),
    datetime(2022, 5, 1): datetime(2022, 6, 10),
    datetime(2022, 6, 1): datetime(2022, 7, 13),
    datetime(2022, 7, 1): datetime(2022, 8, 10),
    datetime(2022, 8, 1): datetime(2022, 9, 13),
    datetime(2022, 9, 1): datetime(2022, 10, 13),
    datetime(2022, 10, 1): datetime(2022, 11, 10),
    datetime(2022, 11, 1): datetime(2022, 12, 13),

    # --- 2023 ---
    datetime(2022, 12, 1): datetime(2023, 1, 12),
    datetime(2023, 1, 1): datetime(2023, 2, 14),
    datetime(2023, 2, 1): datetime(2023, 3, 14),
    datetime(2023, 3, 1): datetime(2023, 4, 12),
    datetime(2023, 4, 1): datetime(2023, 5, 10),
    datetime(2023, 5, 1): datetime(2023, 6, 13),
    datetime(2023, 6, 1): datetime(2023, 7, 12),
    datetime(2023, 7, 1): datetime(2023, 8, 10),
    datetime(2023, 8, 1): datetime(2023, 9, 13),
    datetime(2023, 9, 1): datetime(2023, 10, 12),
    datetime(2023, 10, 1): datetime(2023, 11, 14),
    datetime(2023, 11, 1): datetime(2023, 12, 12),

    # --- 2024 ---
    datetime(2023, 12, 1): datetime(2024, 1, 11),
    datetime(2024, 1, 1): datetime(2024, 2, 13),
    datetime(2024, 2, 1): datetime(2024, 3, 12),
    datetime(2024, 3, 1): datetime(2024, 4, 10),
    datetime(2024, 4, 1): datetime(2024, 5, 15),
    datetime(2024, 5, 1): datetime(2024, 6, 12),
    datetime(2024, 6, 1): datetime(2024, 7, 11),
    datetime(2024, 7, 1): datetime(2024, 8, 14),
    datetime(2024, 8, 1): datetime(2024, 9, 11),
    datetime(2024, 9, 1): datetime(2024, 10, 10),
    datetime(2024, 10, 1): datetime(2024, 11, 13),
    datetime(2024, 11, 1): datetime(2024, 12, 11),

    # --- 2025 ---
    datetime(2024, 12, 1): datetime(2025, 1, 15),
    datetime(2025, 1, 1): datetime(2025, 2, 12),
    datetime(2025, 2, 1): datetime(2025, 3, 12),
    datetime(2025, 3, 1): datetime(2025, 4, 10),
    datetime(2025, 4, 1): datetime(2025, 5, 13),
    datetime(2025, 5, 1): datetime(2025, 6, 11),
    datetime(2025, 6, 1): datetime(2025, 7, 15),
    datetime(2025, 7, 1): datetime(2025, 8, 12),
    datetime(2025, 8, 1): datetime(2025, 9, 11),
    datetime(2025, 9, 1): datetime(2025, 10, 15),
}

In [389]:
# Use the .map() function to apply the dictionary mapping.
cpi_df['release_date'] = cpi_df.index.map(CPI_RELEASE_MAP)

# Check the head()
cpi_df.head()

C:\Users\yehua\AppData\Local\Temp\ipykernel_34236\2036946967.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cpi_df['release_date'] = cpi_df.index.map(CPI_RELEASE_MAP)


,CPI,CPI_yoy_change,release_date
2018-09-01,252.182,2.332055,2018-10-11
2018-10-01,252.772,2.492032,2018-11-14
2018-11-01,252.594,2.147329,2018-12-11
2018-12-01,252.767,2.002381,2019-01-11
2019-01-01,252.561,1.487589,2019-02-13


Map the monthly data into daily data

In [390]:
# Set the release_date as the new index
cpi_daily_df = cpi_df.set_index('release_date')[['CPI', 'CPI_yoy_change']]

cpi_daily_df.head()

,CPI,CPI_yoy_change
release_date,,
2018-10-11,252.182,2.332055
2018-11-14,252.772,2.492032
2018-12-11,252.594,2.147329
2019-01-11,252.767,2.002381
2019-02-13,252.561,1.487589


In [391]:
# Create a complete daily index and forward fill the data
start_date = '2018-09-01'
end_date = '2025-10-22'

new_daily_index = pd.date_range(start=start_date, end=end_date, freq='D', name='Date')

In [392]:
# Reindex the data to the daily frequency
cpi_daily_df = cpi_daily_df.reindex(new_daily_index)

# Check data around announcement date
cpi_daily_df.loc['2018-12-08': '2018-12-13', :]

,CPI,CPI_yoy_change
Date,,
2018-12-08,NaN,NaN
2018-12-09,NaN,NaN
2018-12-10,NaN,NaN
2018-12-11,252.594,2.147329
2018-12-12,NaN,NaN
2018-12-13,NaN,NaN


In [393]:
# Forward fill the values
cpi_daily_df = cpi_daily_df.ffill()

# Check data around announcement date
cpi_daily_df.loc['2018-12-08': '2018-12-13', :]

,CPI,CPI_yoy_change
Date,,
2018-12-08,252.772,2.492032
2018-12-09,252.772,2.492032
2018-12-10,252.772,2.492032
2018-12-11,252.594,2.147329
2018-12-12,252.594,2.147329
2018-12-13,252.594,2.147329


Filter the cpi daily data to store the data that is in the yield_changes_df

In [394]:
# Get the index (trading days from 2019 to 2025)
trading_dates = yield_changes_df.index

# Filter the cpi_daily_df
cpi_daily_df = cpi_daily_df[cpi_daily_df.index.isin(trading_dates)]

# Check the head
cpi_daily_df.head(8)

,CPI,CPI_yoy_change
Date,,
2019-01-03,252.594,2.147329
2019-01-04,252.594,2.147329
2019-01-07,252.594,2.147329
2019-01-08,252.594,2.147329
2019-01-09,252.594,2.147329
2019-01-10,252.594,2.147329
2019-01-11,252.767,2.002381
2019-01-14,252.767,2.002381


In [395]:
# Get the updated shape of cpi_daily_df
cpi_daily_df.shape

(1702, 2)

---

### 5.2 PPI <a class="anchor" id="5b"></a>

In [396]:
# ---  Define the Official PPI Release Date Mapping (BLS Schedule) ---
# Key: The first day of the data's reference month (e.g., '2019-01-01' for Jan 2019 data)
# Value: The official BLS release date

PPI_RELEASE_MAP = {

    # 2017 
    datetime(2017, 9, 1): datetime(2017, 10, 13),
    datetime(2017, 10, 1): datetime(2017, 11, 14),
    datetime(2017, 11, 1): datetime(2017, 12, 12),

    # 2018 
    datetime(2017, 12, 1): datetime(2018, 1, 11),
    datetime(2018, 1, 1): datetime(2018, 2, 15),
    datetime(2018, 2, 1): datetime(2018, 3, 14),
    datetime(2018, 3, 1): datetime(2018, 4, 12),
    datetime(2018, 4, 1): datetime(2018, 5, 15),
    datetime(2018, 5, 1): datetime(2018, 6, 13),
    datetime(2018, 6, 1): datetime(2018, 7, 12),
    datetime(2018, 7, 1): datetime(2018, 8, 14),
    datetime(2018, 8, 1): datetime(2018, 9, 12),
    datetime(2018, 9, 1): datetime(2018, 10, 10),
    datetime(2018, 10, 1): datetime(2018, 11, 13),
    datetime(2018, 11, 1): datetime(2018, 12, 11), 
    datetime(2018, 12, 1): datetime(2019, 1, 15),

    # 2019
    datetime(2019, 1, 1): datetime(2019, 2, 14),
    datetime(2019, 2, 1): datetime(2019, 3, 13),
    datetime(2019, 3, 1): datetime(2019, 4, 11),
    datetime(2019, 4, 1): datetime(2019, 5, 14),
    datetime(2019, 5, 1): datetime(2019, 6, 11),
    datetime(2019, 6, 1): datetime(2019, 7, 12),
    datetime(2019, 7, 1): datetime(2019, 8, 13),
    datetime(2019, 8, 1): datetime(2019, 9, 11),
    datetime(2019, 9, 1): datetime(2019, 10, 15),
    datetime(2019, 10, 1): datetime(2019, 11, 13),
    datetime(2019, 11, 1): datetime(2019, 12, 11),

    # 2020
    datetime(2019, 12, 1): datetime(2020, 1, 14),
    datetime(2020, 1, 1): datetime(2020, 2, 19),
    datetime(2020, 2, 1): datetime(2020, 3, 17),
    datetime(2020, 3, 1): datetime(2020, 4, 15),
    datetime(2020, 4, 1): datetime(2020, 5, 12),
    datetime(2020, 5, 1): datetime(2020, 6, 10),
    datetime(2020, 6, 1): datetime(2020, 7, 10),
    datetime(2020, 7, 1): datetime(2020, 8, 11),
    datetime(2020, 8, 1): datetime(2020, 9, 11),
    datetime(2020, 9, 1): datetime(2020, 10, 14),
    datetime(2020, 10, 1): datetime(2020, 11, 13),
    datetime(2020, 11, 1): datetime(2020, 12, 11),

    # 2021
    datetime(2020, 12, 1): datetime(2021, 1, 15),
    datetime(2021, 1, 1): datetime(2021, 2, 17),
    datetime(2021, 2, 1): datetime(2021, 3, 12),
    datetime(2021, 3, 1): datetime(2021, 4, 9),
    datetime(2021, 4, 1): datetime(2021, 5, 14),
    datetime(2021, 5, 1): datetime(2021, 6, 15),
    datetime(2021, 6, 1): datetime(2021, 7, 16),
    datetime(2021, 7, 1): datetime(2021, 8, 12),
    datetime(2021, 8, 1): datetime(2021, 9, 10),
    datetime(2021, 9, 1): datetime(2021, 10, 14),
    datetime(2021, 10, 1): datetime(2021, 11, 16),
    datetime(2021, 11, 1): datetime(2021, 12, 14),

    # 2022
    datetime(2021, 12, 1): datetime(2022, 1, 13),
    datetime(2022, 1, 1): datetime(2022, 2, 15),
    datetime(2022, 2, 1): datetime(2022, 3, 15),
    datetime(2022, 3, 1): datetime(2022, 4, 13),
    datetime(2022, 4, 1): datetime(2022, 5, 12),
    datetime(2022, 5, 1): datetime(2022, 6, 14),
    datetime(2022, 6, 1): datetime(2022, 7, 14),
    datetime(2022, 7, 1): datetime(2022, 8, 18),
    datetime(2022, 8, 1): datetime(2022, 9, 14),
    datetime(2022, 9, 1): datetime(2022, 10, 12),
    datetime(2022, 10, 1): datetime(2022, 11, 15),
    datetime(2022, 11, 1): datetime(2022, 12, 9),

    # 2023
    datetime(2022, 12, 1): datetime(2023, 1, 18),
    datetime(2023, 1, 1): datetime(2023, 2, 16),
    datetime(2023, 2, 1): datetime(2023, 3, 15),
    datetime(2023, 3, 1): datetime(2023, 4, 13),
    datetime(2023, 4, 1): datetime(2023, 5, 11),
    datetime(2023, 5, 1): datetime(2023, 6, 14),
    datetime(2023, 6, 1): datetime(2023, 7, 13),
    datetime(2023, 7, 1): datetime(2023, 8, 11),
    datetime(2023, 8, 1): datetime(2023, 9, 14),
    datetime(2023, 9, 1): datetime(2023, 10, 11),
    datetime(2023, 10, 1): datetime(2023, 11, 15),
    datetime(2023, 11, 1): datetime(2023, 12, 13),

    # 2024
    datetime(2023, 12, 1): datetime(2024, 1, 12),
    datetime(2024, 1, 1): datetime(2024, 2, 16),
    datetime(2024, 2, 1): datetime(2024, 3, 14),
    datetime(2024, 3, 1): datetime(2024, 4, 11),
    datetime(2024, 4, 1): datetime(2024, 5, 14),
    datetime(2024, 5, 1): datetime(2024, 6, 13),
    datetime(2024, 6, 1): datetime(2024, 7, 12),
    datetime(2024, 7, 1): datetime(2024, 8, 13),
    datetime(2024, 8, 1): datetime(2024, 9, 12),
    datetime(2024, 9, 1): datetime(2024, 10, 11),
    datetime(2024, 10, 1): datetime(2024, 11, 14),
    datetime(2024, 11, 1): datetime(2024, 12, 12),

    # 2025 (Up to September data)
    datetime(2024, 12, 1): datetime(2025, 1, 14),
    datetime(2025, 1, 1): datetime(2025, 2, 13),
    datetime(2025, 2, 1): datetime(2025, 3, 13),
    datetime(2025, 3, 1): datetime(2025, 4, 11),
    datetime(2025, 4, 1): datetime(2025, 5, 15),
    datetime(2025, 5, 1): datetime(2025, 6, 12),
    datetime(2025, 6, 1): datetime(2025, 7, 16),
    datetime(2025, 7, 1): datetime(2025, 8, 14),
    datetime(2025, 8, 1): datetime(2025, 9, 10),
    datetime(2025, 9, 1): datetime(2025, 10, 16),   
}

In [397]:
#  Map the Reference Date to the Release Date 
ppi_df['release_date'] = ppi_df.index.map(PPI_RELEASE_MAP)

C:\Users\yehua\AppData\Local\Temp\ipykernel_34236\2314835768.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppi_df['release_date'] = ppi_df.index.map(PPI_RELEASE_MAP)


In [398]:
# Check the head
ppi_df.head()

,PPI,PPI_yoy_change,release_date
2018-09-01,203.6,4.517454,2018-10-10
2018-10-01,204.6,4.976911,2018-11-13
2018-11-01,202.3,3.266973,2018-12-11
2018-12-01,201.0,2.394294,2019-01-15
2019-01-01,199.1,0.606367,2019-02-14


Map the monthly data into daily data

In [399]:
# Set the release_date as the new index
ppi_daily_df = ppi_df.set_index('release_date')[['PPI', 'PPI_yoy_change']]

# Create a new daily index
start_date = '2018-09-01'
end_date = '2025-10-22'
new_daily_index = pd.date_range(start=start_date, end=end_date, freq='D', name='Date')

# Reindex the data to the daily frequency
ppi_daily_df = ppi_daily_df.reindex(new_daily_index)

# Check data around announcement date
ppi_daily_df.loc['2018-12-08': '2018-12-13', :]

,PPI,PPI_yoy_change
Date,,
2018-12-08,NaN,NaN
2018-12-09,NaN,NaN
2018-12-10,NaN,NaN
2018-12-11,202.3,3.266973
2018-12-12,NaN,NaN
2018-12-13,NaN,NaN


In [400]:
# Forward fill the values
ppi_daily_df = ppi_daily_df.ffill()

# Check data around announcement date
ppi_daily_df.loc['2018-12-08': '2018-12-13', :]

,PPI,PPI_yoy_change
Date,,
2018-12-08,204.6,4.976911
2018-12-09,204.6,4.976911
2018-12-10,204.6,4.976911
2018-12-11,202.3,3.266973
2018-12-12,202.3,3.266973
2018-12-13,202.3,3.266973


Filter the ppi daily data to store the data that is in the yield_changes_df

In [401]:
# Filter the ppi_daily_df
ppi_daily_df = ppi_daily_df[ppi_daily_df.index.isin(trading_dates)]

# Check the head
ppi_daily_df.head(11)

,PPI,PPI_yoy_change
Date,,
2019-01-03,202.3,3.266973
2019-01-04,202.3,3.266973
2019-01-07,202.3,3.266973
2019-01-08,202.3,3.266973
2019-01-09,202.3,3.266973
2019-01-10,202.3,3.266973
2019-01-11,202.3,3.266973
2019-01-14,202.3,3.266973
2019-01-15,201.0,2.394294


In [402]:
# Get the updated shape
ppi_daily_df.shape

(1702, 2)

---

### 5.3 PCE <a class="anchor" id="5c"></a>

In [403]:
# --- Define the Official PCE Release Date Mapping (BEA Schedule) ---
# Key: The first day of the data's reference month (e.g., '2019-01-01' for Jan 2019 data)
# Value: The official BEA release date (Personal Income and Outlays report)

PCE_RELEASE_MAP = {
    # 2017 
    datetime(2017, 9, 1): datetime(2017, 10, 30),
    datetime(2017, 10, 1): datetime(2017, 11, 30),
    datetime(2017, 11, 1): datetime(2017, 12, 22),

    # 2018 
    datetime(2017, 12, 1): datetime(2018, 1, 29),
    datetime(2018, 1, 1): datetime(2018, 3, 1),
    datetime(2018, 2, 1): datetime(2018, 3, 29),
    datetime(2018, 3, 1): datetime(2018, 4, 30),
    datetime(2018, 4, 1): datetime(2018, 5, 29),
    datetime(2018, 5, 1): datetime(2018, 6, 29),
    datetime(2018, 6, 1): datetime(2018, 7, 30),
    datetime(2018, 7, 1): datetime(2018, 8, 30),
    datetime(2018, 8, 1): datetime(2018, 9, 28),
    datetime(2018, 9, 1): datetime(2018, 10, 29),
    datetime(2018, 10, 1): datetime(2018, 11, 30),
    datetime(2018, 11, 1): datetime(2018, 12, 21),

    # 2019 
    datetime(2018, 12, 1): datetime(2019, 1, 31), 
    datetime(2019, 1, 1): datetime(2019, 3, 1),
    datetime(2019, 2, 1): datetime(2019, 3, 29),
    datetime(2019, 3, 1): datetime(2019, 4, 29),
    datetime(2019, 4, 1): datetime(2019, 5, 31),
    datetime(2019, 5, 1): datetime(2019, 6, 28),
    datetime(2019, 6, 1): datetime(2019, 7, 30),
    datetime(2019, 7, 1): datetime(2019, 8, 30),
    datetime(2019, 8, 1): datetime(2019, 9, 27),
    datetime(2019, 9, 1): datetime(2019, 10, 31),
    datetime(2019, 10, 1): datetime(2019, 11, 27),
    datetime(2019, 11, 1): datetime(2019, 12, 20),

    # 2020
    datetime(2019, 12, 1): datetime(2020, 1, 31),
    datetime(2020, 1, 1): datetime(2020, 2, 28),
    datetime(2020, 2, 1): datetime(2020, 3, 27),
    datetime(2020, 3, 1): datetime(2020, 4, 30),
    datetime(2020, 4, 1): datetime(2020, 5, 29),
    datetime(2020, 5, 1): datetime(2020, 6, 26),
    datetime(2020, 6, 1): datetime(2020, 7, 31),
    datetime(2020, 7, 1): datetime(2020, 8, 28),
    datetime(2020, 8, 1): datetime(2020, 9, 25),
    datetime(2020, 9, 1): datetime(2020, 10, 30),
    datetime(2020, 10, 1): datetime(2020, 11, 25),
    datetime(2020, 11, 1): datetime(2020, 12, 23),

    # 2021
    datetime(2020, 12, 1): datetime(2021, 1, 29),
    datetime(2021, 1, 1): datetime(2021, 2, 26),
    datetime(2021, 2, 1): datetime(2021, 3, 26),
    datetime(2021, 3, 1): datetime(2021, 4, 30),
    datetime(2021, 4, 1): datetime(2021, 5, 28),
    datetime(2021, 5, 1): datetime(2021, 6, 25),
    datetime(2021, 6, 1): datetime(2021, 7, 30),
    datetime(2021, 7, 1): datetime(2021, 8, 27),
    datetime(2021, 8, 1): datetime(2021, 9, 27),
    datetime(2021, 9, 1): datetime(2021, 10, 29),
    datetime(2021, 10, 1): datetime(2021, 11, 24),
    datetime(2021, 11, 1): datetime(2021, 12, 23),

    # 2022
    datetime(2021, 12, 1): datetime(2022, 1, 28),
    datetime(2022, 1, 1): datetime(2022, 2, 25),
    datetime(2022, 2, 1): datetime(2022, 3, 31),
    datetime(2022, 3, 1): datetime(2022, 4, 29),
    datetime(2022, 4, 1): datetime(2022, 5, 27),
    datetime(2022, 5, 1): datetime(2022, 6, 30),
    datetime(2022, 6, 1): datetime(2022, 7, 29),
    datetime(2022, 7, 1): datetime(2022, 8, 26),
    datetime(2022, 8, 1): datetime(2022, 9, 30),
    datetime(2022, 9, 1): datetime(2022, 10, 28),
    datetime(2022, 10, 1): datetime(2022, 11, 30),
    datetime(2022, 11, 1): datetime(2022, 12, 23),

    # 2023
    datetime(2022, 12, 1): datetime(2023, 1, 27),
    datetime(2023, 1, 1): datetime(2023, 2, 24),
    datetime(2023, 2, 1): datetime(2023, 3, 31),
    datetime(2023, 3, 1): datetime(2023, 4, 28),
    datetime(2023, 4, 1): datetime(2023, 5, 26),
    datetime(2023, 5, 1): datetime(2023, 6, 30),
    datetime(2023, 6, 1): datetime(2023, 7, 28),
    datetime(2023, 7, 1): datetime(2023, 8, 31),
    datetime(2023, 8, 1): datetime(2023, 9, 29),
    datetime(2023, 9, 1): datetime(2023, 10, 27),
    datetime(2023, 10, 1): datetime(2023, 11, 30),
    datetime(2023, 11, 1): datetime(2023, 12, 22),

    # 2024
    datetime(2023, 12, 1): datetime(2024, 1, 26),
    datetime(2024, 1, 1): datetime(2024, 3, 1),
    datetime(2024, 2, 1): datetime(2024, 3, 29),
    datetime(2024, 3, 1): datetime(2024, 4, 26),
    datetime(2024, 4, 1): datetime(2024, 5, 31),
    datetime(2024, 5, 1): datetime(2024, 6, 28),
    datetime(2024, 6, 1): datetime(2024, 7, 26),
    datetime(2024, 7, 1): datetime(2024, 8, 30),
    datetime(2024, 8, 1): datetime(2024, 9, 27),
    datetime(2024, 9, 1): datetime(2024, 10, 31),
    datetime(2024, 10, 1): datetime(2024, 11, 27),
    datetime(2024, 11, 1): datetime(2024, 12, 20),

    # 2025 (Up to September data)
    datetime(2024, 12, 1): datetime(2025, 1, 31),
    datetime(2025, 1, 1): datetime(2025, 2, 28),
    datetime(2025, 2, 1): datetime(2025, 3, 28),
    datetime(2025, 3, 1): datetime(2025, 4, 30),
    datetime(2025, 4, 1): datetime(2025, 5, 30),
    datetime(2025, 5, 1): datetime(2025, 6, 27),
    datetime(2025, 6, 1): datetime(2025, 7, 31),
    datetime(2025, 7, 1): datetime(2025, 8, 29),
    datetime(2025, 8, 1): datetime(2025, 9, 26),
    datetime(2025, 9, 1): datetime(2025, 10, 31),
}

In [404]:
#  Map the Reference Date to the Release Date 
pce_df['release_date'] = pce_df.index.map(PCE_RELEASE_MAP)

C:\Users\yehua\AppData\Local\Temp\ipykernel_34236\2045513023.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pce_df['release_date'] = pce_df.index.map(PCE_RELEASE_MAP)


In [405]:
# Check the head
pce_df.head()

,PCE,PCE_yoy_change,release_date
2018-09-01,102.418,2.009960,2018-10-29
2018-10-01,102.595,2.053098,2018-11-30
2018-11-01,102.630,1.941892,2018-12-21
2018-12-01,102.701,1.864691,2019-01-31
2019-01-01,102.648,1.431832,2019-03-01


Map the monthly data into daily data

In [406]:
# Set the release_date as the new index
pce_daily_df = pce_df.set_index('release_date')[['PCE', 'PCE_yoy_change']]

# Create a new daily index
start_date = '2018-09-01'
end_date = '2025-10-22'
new_daily_index = pd.date_range(start=start_date, end=end_date, freq='D', name='Date')

# Reindex the data to the daily frequency
pce_daily_df = pce_daily_df.reindex(new_daily_index)

# Check data around announcement date
pce_daily_df.loc['2018-12-18': '2018-12-22', :]

,PCE,PCE_yoy_change
Date,,
2018-12-18,NaN,NaN
2018-12-19,NaN,NaN
2018-12-20,NaN,NaN
2018-12-21,102.63,1.941892
2018-12-22,NaN,NaN


In [407]:
# Forward fill the values
pce_daily_df = pce_daily_df.ffill()

# Check data around announcement date
pce_daily_df.loc['2018-12-18': '2018-12-22', :]

,PCE,PCE_yoy_change
Date,,
2018-12-18,102.595,2.053098
2018-12-19,102.595,2.053098
2018-12-20,102.595,2.053098
2018-12-21,102.630,1.941892
2018-12-22,102.630,1.941892


Filter the pce daily data to store the data that is in the yield_changes_df

In [408]:
# Filter the pce_daily_df
pce_daily_df = pce_daily_df[pce_daily_df.index.isin(trading_dates)]

# Check the data
pce_daily_df.iloc[15:21, :]

,PCE,PCE_yoy_change
Date,,
2019-01-25,102.630,1.941892
2019-01-28,102.630,1.941892
2019-01-29,102.630,1.941892
2019-01-30,102.630,1.941892
2019-01-31,102.701,1.864691
2019-02-01,102.701,1.864691


---

### 5.4 Unemployment Rate <a class="anchor" id="5d"></a>

In [409]:
# ---  Define the Official Unemployment Rate Release Date Map (BLS Employment Situation Schedule) ---
# Key: The first day of the data's reference month (e.g., '2019-01-01' for Jan 2019 data)
# Value: The official BLS release date (First Friday of the following month, at 8:30 AM ET)

UNEMPLOYMENT_RATE_RELEASE_MAP = {
    # 2017 
    datetime(2017, 9, 1): datetime(2017, 10, 6),
    datetime(2017, 10, 1): datetime(2017, 11, 3),
    datetime(2017, 11, 1): datetime(2017, 12, 8), 

    # 2018 
    datetime(2017, 12, 1): datetime(2018, 1, 5),
    datetime(2018, 1, 1): datetime(2018, 2, 2),
    datetime(2018, 2, 1): datetime(2018, 3, 9),
    datetime(2018, 3, 1): datetime(2018, 4, 6),
    datetime(2018, 4, 1): datetime(2018, 5, 4),
    datetime(2018, 5, 1): datetime(2018, 6, 1),
    datetime(2018, 6, 1): datetime(2018, 7, 6),
    datetime(2018, 7, 1): datetime(2018, 8, 3),
    datetime(2018, 8, 1): datetime(2018, 9, 7),
    datetime(2018, 9, 1): datetime(2018, 10, 5),
    datetime(2018, 10, 1): datetime(2018, 11, 2),
    datetime(2018, 11, 1): datetime(2018, 12, 7),

    # 2019 
    datetime(2018, 12, 1): datetime(2019, 1, 4), 
    datetime(2019, 1, 1): datetime(2019, 2, 1),
    datetime(2019, 2, 1): datetime(2019, 3, 8),
    datetime(2019, 3, 1): datetime(2019, 4, 5),
    datetime(2019, 4, 1): datetime(2019, 5, 3),
    datetime(2019, 5, 1): datetime(2019, 6, 7),
    datetime(2019, 6, 1): datetime(2019, 7, 5),
    datetime(2019, 7, 1): datetime(2019, 8, 2),
    datetime(2019, 8, 1): datetime(2019, 9, 6),
    datetime(2019, 9, 1): datetime(2019, 10, 4),
    datetime(2019, 10, 1): datetime(2019, 11, 8),
    datetime(2019, 11, 1): datetime(2019, 12, 6),

    # 2020
    datetime(2019, 12, 1): datetime(2020, 1, 10),
    datetime(2020, 1, 1): datetime(2020, 2, 7),
    datetime(2020, 2, 1): datetime(2020, 3, 6),
    datetime(2020, 3, 1): datetime(2020, 4, 3),
    datetime(2020, 4, 1): datetime(2020, 5, 8),
    datetime(2020, 5, 1): datetime(2020, 6, 5),
    datetime(2020, 6, 1): datetime(2020, 7, 2),
    datetime(2020, 7, 1): datetime(2020, 8, 7),
    datetime(2020, 8, 1): datetime(2020, 9, 4),
    datetime(2020, 9, 1): datetime(2020, 10, 2),
    datetime(2020, 10, 1): datetime(2020, 11, 6),
    datetime(2020, 11, 1): datetime(2020, 12, 4),

    # 2021
    datetime(2020, 12, 1): datetime(2021, 1, 8),
    datetime(2021, 1, 1): datetime(2021, 2, 5),
    datetime(2021, 2, 1): datetime(2021, 3, 5),
    datetime(2021, 3, 1): datetime(2021, 4, 2),
    datetime(2021, 4, 1): datetime(2021, 5, 7),
    datetime(2021, 5, 1): datetime(2021, 6, 4),
    datetime(2021, 6, 1): datetime(2021, 7, 2),
    datetime(2021, 7, 1): datetime(2021, 8, 6),
    datetime(2021, 8, 1): datetime(2021, 9, 3),
    datetime(2021, 9, 1): datetime(2021, 10, 8),
    datetime(2021, 10, 1): datetime(2021, 11, 5),
    datetime(2021, 11, 1): datetime(2021, 12, 3),

    # 2022
    datetime(2021, 12, 1): datetime(2022, 1, 7),
    datetime(2022, 1, 1): datetime(2022, 2, 4),
    datetime(2022, 2, 1): datetime(2022, 3, 4),
    datetime(2022, 3, 1): datetime(2022, 4, 1),
    datetime(2022, 4, 1): datetime(2022, 5, 6),
    datetime(2022, 5, 1): datetime(2022, 6, 3),
    datetime(2022, 6, 1): datetime(2022, 7, 8),
    datetime(2022, 7, 1): datetime(2022, 8, 5),
    datetime(2022, 8, 1): datetime(2022, 9, 2),
    datetime(2022, 9, 1): datetime(2022, 10, 7),
    datetime(2022, 10, 1): datetime(2022, 11, 4),
    datetime(2022, 11, 1): datetime(2022, 12, 2),

    # 2023
    datetime(2022, 12, 1): datetime(2023, 1, 6),
    datetime(2023, 1, 1): datetime(2023, 2, 3),
    datetime(2023, 2, 1): datetime(2023, 3, 10),
    datetime(2023, 3, 1): datetime(2023, 4, 7),
    datetime(2023, 4, 1): datetime(2023, 5, 5),
    datetime(2023, 5, 1): datetime(2023, 6, 2),
    datetime(2023, 6, 1): datetime(2023, 7, 7),
    datetime(2023, 7, 1): datetime(2023, 8, 4),
    datetime(2023, 8, 1): datetime(2023, 9, 1),
    datetime(2023, 9, 1): datetime(2023, 10, 6),
    datetime(2023, 10, 1): datetime(2023, 11, 3),
    datetime(2023, 11, 1): datetime(2023, 12, 8),

    # 2024
    datetime(2023, 12, 1): datetime(2024, 1, 5),
    datetime(2024, 1, 1): datetime(2024, 2, 2),
    datetime(2024, 2, 1): datetime(2024, 3, 8),
    datetime(2024, 3, 1): datetime(2024, 4, 5),
    datetime(2024, 4, 1): datetime(2024, 5, 3),
    datetime(2024, 5, 1): datetime(2024, 6, 7),
    datetime(2024, 6, 1): datetime(2024, 7, 5),
    datetime(2024, 7, 1): datetime(2024, 8, 2),
    datetime(2024, 8, 1): datetime(2024, 9, 6),
    datetime(2024, 9, 1): datetime(2024, 10, 4),
    datetime(2024, 10, 1): datetime(2024, 11, 1),
    datetime(2024, 11, 1): datetime(2024, 12, 6),

    # 2025 (Up to September data)
    datetime(2024, 12, 1): datetime(2025, 1, 10),
    datetime(2025, 1, 1): datetime(2025, 2, 7),
    datetime(2025, 2, 1): datetime(2025, 3, 7),
    datetime(2025, 3, 1): datetime(2025, 4, 4),
    datetime(2025, 4, 1): datetime(2025, 5, 2),
    datetime(2025, 5, 1): datetime(2025, 6, 6),
    datetime(2025, 6, 1): datetime(2025, 7, 3),
    datetime(2025, 7, 1): datetime(2025, 8, 1),
    datetime(2025, 8, 1): datetime(2025, 9, 5),
    datetime(2025, 9, 1): datetime(2025, 10, 3),
}

In [410]:
#  Map the Reference Date to the Release Date 
u_rate_df['release_date'] = u_rate_df.index.map(UNEMPLOYMENT_RATE_RELEASE_MAP)

C:\Users\yehua\AppData\Local\Temp\ipykernel_34236\925394460.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  u_rate_df['release_date'] = u_rate_df.index.map(UNEMPLOYMENT_RATE_RELEASE_MAP)


In [411]:
# Check the head
u_rate_df.head()

,Unemployment Rate,Unemployment_mom_change,Unemployment_yoy_change,release_date
2018-09-01,3.7,-2.631579,-13.953488,2018-10-05
2018-10-01,3.8,2.702703,-9.523810,2018-11-02
2018-11-01,3.8,0.000000,-9.523810,2018-12-07
2018-12-01,3.9,2.631579,-4.878049,2019-01-04
2019-01-01,4.0,2.564103,0.000000,2019-02-01


Map the monthly data into daily data

In [412]:
# Set the release_date as the new index
u_rate_daily_df = u_rate_df.set_index('release_date')[['Unemployment Rate', 'Unemployment_mom_change', 'Unemployment_yoy_change']]

# Create a new daily index
start_date = '2018-09-01'
end_date = '2025-10-22'
new_daily_index = pd.date_range(start=start_date, end=end_date, freq='D', name='Date')

# Reindex the data to the daily frequency
u_rate_daily_df = u_rate_daily_df.reindex(new_daily_index)

# Check data around announcement date
u_rate_daily_df.loc['2018-12-05': '2018-12-10', :]

,Unemployment Rate,Unemployment_mom_change,Unemployment_yoy_change
Date,,,
2018-12-05,NaN,NaN,NaN
2018-12-06,NaN,NaN,NaN
2018-12-07,3.8,0.0,-9.52381
2018-12-08,NaN,NaN,NaN
2018-12-09,NaN,NaN,NaN
2018-12-10,NaN,NaN,NaN


In [413]:
# Forward fill the values
u_rate_daily_df = u_rate_daily_df.ffill()

# Check data around announcement date
u_rate_daily_df.loc['2018-12-05': '2018-12-10', :]

,Unemployment Rate,Unemployment_mom_change,Unemployment_yoy_change
Date,,,
2018-12-05,3.8,2.702703,-9.52381
2018-12-06,3.8,2.702703,-9.52381
2018-12-07,3.8,0.000000,-9.52381
2018-12-08,3.8,0.000000,-9.52381
2018-12-09,3.8,0.000000,-9.52381
2018-12-10,3.8,0.000000,-9.52381


Filter the u_rate_daily_df data to store the data that is in the yield_changes_df

In [414]:
# Filter the ppi_daily_df
u_rate_daily_df = u_rate_daily_df[u_rate_daily_df.index.isin(trading_dates)]

# Check the head
u_rate_daily_df.head(3)

,Unemployment Rate,Unemployment_mom_change,Unemployment_yoy_change
Date,,,
2019-01-03,3.8,0.000000,-9.523810
2019-01-04,3.9,2.631579,-4.878049
2019-01-07,3.9,2.631579,-4.878049


---

### 5.5 GDP <a class="anchor" id="5e"></a>

In [415]:
# ---  Define the Official Advance Estimate GDP Release Date Map (BEA Schedule) ---
# Key: The first day of the data's reference quarter (e.g., '2019-01-01' for Q1 2019 data)
# Value: The official BEA Advance Estimate Release Date (approximate last week of the following month)

GDP_RELEASE_MAP = {
    # 2017
    datetime(2017, 1, 1): datetime(2017, 4, 28), 
    datetime(2017, 4, 1): datetime(2017, 7, 28), 
    datetime(2017, 7, 1): datetime(2017, 10, 27), 
    datetime(2017, 10, 1): datetime(2018, 1, 26), 

    # 2018
    datetime(2018, 1, 1): datetime(2018, 4, 27), 
    datetime(2018, 4, 1): datetime(2018, 7, 27), 
    datetime(2018, 7, 1): datetime(2018, 10, 26), 
    datetime(2018, 10, 1): datetime(2019, 1, 30), 

    # 2019 (Original Data starts here)
    datetime(2019, 1, 1): datetime(2019, 4, 26),
    datetime(2019, 4, 1): datetime(2019, 7, 26),
    datetime(2019, 7, 1): datetime(2019, 10, 30),
    datetime(2019, 10, 1): datetime(2020, 1, 30),

    # 2020
    datetime(2020, 1, 1): datetime(2020, 4, 29),
    datetime(2020, 4, 1): datetime(2020, 7, 30),
    datetime(2020, 7, 1): datetime(2020, 10, 29),
    datetime(2020, 10, 1): datetime(2021, 1, 28),

    # 2021
    datetime(2021, 1, 1): datetime(2021, 4, 29),
    datetime(2021, 4, 1): datetime(2021, 7, 29),
    datetime(2021, 7, 1): datetime(2021, 10, 28),
    datetime(2021, 10, 1): datetime(2022, 1, 27),

    # 2022
    datetime(2022, 1, 1): datetime(2022, 4, 28),
    datetime(2022, 4, 1): datetime(2022, 7, 28),
    datetime(2022, 7, 1): datetime(2022, 10, 27),
    datetime(2022, 10, 1): datetime(2023, 1, 26),

    # 2023
    datetime(2023, 1, 1): datetime(2023, 4, 27),
    datetime(2023, 4, 1): datetime(2023, 7, 27),
    datetime(2023, 7, 1): datetime(2023, 10, 26),
    datetime(2023, 10, 1): datetime(2024, 1, 25),

    # 2024
    datetime(2024, 1, 1): datetime(2024, 4, 25),
    datetime(2024, 4, 1): datetime(2024, 7, 25),
    datetime(2024, 7, 1): datetime(2024, 10, 30),
    datetime(2024, 10, 1): datetime(2025, 1, 30),

    # 2025 (Up to Q3 data)
    datetime(2025, 1, 1): datetime(2025, 4, 30),
    datetime(2025, 4, 1): datetime(2025, 7, 30),
    datetime(2025, 7, 1): datetime(2025, 10, 30)
}

In [416]:
#  Map the Reference Date to the Release Date 
gdp_df['release_date'] = gdp_df.index.map(GDP_RELEASE_MAP)

C:\Users\yehua\AppData\Local\Temp\ipykernel_34236\663080920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdp_df['release_date'] = gdp_df.index.map(GDP_RELEASE_MAP)


In [417]:
# Check the head
gdp_df.head()

,GDP,GDP_qoq_change,GDP_yoy_change,release_date
2018-07-01,20798.730,1.058350,5.617010,2018-10-26
2018-10-01,20917.867,0.572809,4.395744,2019-01-30
2019-01-01,21111.600,0.926160,3.851956,2019-04-26
2019-04-01,21397.938,1.356306,3.969824,2019-07-26
2019-07-01,21717.171,1.491887,4.415851,2019-10-30


Map the quarterly data into daily data

In [418]:
# Set the release_date as the new index
gdp_daily_df = gdp_df.set_index('release_date')[['GDP', 'GDP_qoq_change', 'GDP_yoy_change']]

# Create a new daily index
start_date = '2018-09-01'
end_date = '2025-10-22'
new_daily_index = pd.date_range(start=start_date, end=end_date, freq='D', name='Date')

# Reindex the data to the daily frequency
gdp_daily_df = gdp_daily_df.reindex(new_daily_index)

# Check data around announcement date
gdp_daily_df.loc['2019-01-28': '2019-02-02', :]

,GDP,GDP_qoq_change,GDP_yoy_change
Date,,,
2019-01-28,NaN,NaN,NaN
2019-01-29,NaN,NaN,NaN
2019-01-30,20917.867,0.572809,4.395744
2019-01-31,NaN,NaN,NaN
2019-02-01,NaN,NaN,NaN
2019-02-02,NaN,NaN,NaN


In [419]:
# Forward fill the values
gdp_daily_df = gdp_daily_df.ffill()

# Check data around announcement date
gdp_daily_df.loc['2019-01-28': '2019-02-02', :]

,GDP,GDP_qoq_change,GDP_yoy_change
Date,,,
2019-01-28,20798.730,1.058350,5.617010
2019-01-29,20798.730,1.058350,5.617010
2019-01-30,20917.867,0.572809,4.395744
2019-01-31,20917.867,0.572809,4.395744
2019-02-01,20917.867,0.572809,4.395744
2019-02-02,20917.867,0.572809,4.395744


Filter the gdp daily data to store the data that is in the yield_changes_df

In [420]:
# Filter the gdp_daily_df
gdp_daily_df = gdp_daily_df[gdp_daily_df.index.isin(trading_dates)]

# Check the head
gdp_daily_df.head()

,GDP,GDP_qoq_change,GDP_yoy_change
Date,,,
2019-01-03,20798.73,1.05835,5.61701
2019-01-04,20798.73,1.05835,5.61701
2019-01-07,20798.73,1.05835,5.61701
2019-01-08,20798.73,1.05835,5.61701
2019-01-09,20798.73,1.05835,5.61701


In [421]:
# Get the updated shape
gdp_daily_df.shape

(1702, 3)

---

### 5.6 Federal Funds Rate <a class="anchor" id="5f"></a>

We have already obtained Fed Funds Rate in Section 4

In [422]:
# Display Fed Rate Data
fed_rate_df

,Fed_Fund_Rate_Lower,Fed_Fund_Rate_Lower_Upper,rate_change_bps
DATE,,,
2018-12-20,2.25,2.50,25.0
2020-03-16,0.00,0.25,-225.0
2022-03-17,0.25,0.50,25.0
2022-05-05,0.75,1.00,50.0
2022-09-22,3.00,3.25,225.0
2022-11-03,3.75,4.00,75.0
2022-12-15,4.25,4.50,50.0
2023-02-02,4.50,4.75,25.0
2023-03-23,4.75,5.00,25.0


In [423]:
# Reindex the data to the trading day frequency
fed_rate_daily_df = fed_rate_df.reindex(new_daily_index)

# Check data around announcement date
fed_rate_daily_df.loc['2018-12-18': '2018-12-22', :]

,Fed_Fund_Rate_Lower,Fed_Fund_Rate_Lower_Upper,rate_change_bps
Date,,,
2018-12-18,NaN,NaN,NaN
2018-12-19,NaN,NaN,NaN
2018-12-20,2.25,2.5,25.0
2018-12-21,NaN,NaN,NaN
2018-12-22,NaN,NaN,NaN


In [424]:
# Forward fill the data
fed_rate_daily_df = fed_rate_daily_df.ffill()

# Check data around announcement date
fed_rate_daily_df.loc['2018-12-18': '2018-12-22', :]

,Fed_Fund_Rate_Lower,Fed_Fund_Rate_Lower_Upper,rate_change_bps
Date,,,
2018-12-18,NaN,NaN,NaN
2018-12-19,NaN,NaN,NaN
2018-12-20,2.25,2.5,25.0
2018-12-21,2.25,2.5,25.0
2018-12-22,2.25,2.5,25.0


Filter the Fed Rate daily data to store the data that is in the yield_changes_df

In [426]:
# Filter the fed_rate_daily_df
fed_rate_daily_df = fed_rate_daily_df[fed_rate_daily_df.index.isin(trading_dates)]

# Check the head
fed_rate_daily_df.head()

,Fed_Fund_Rate_Lower,Fed_Fund_Rate_Lower_Upper,rate_change_bps
Date,,,
2019-01-03,2.25,2.5,25.0
2019-01-04,2.25,2.5,25.0
2019-01-07,2.25,2.5,25.0
2019-01-08,2.25,2.5,25.0
2019-01-09,2.25,2.5,25.0


In [427]:
# Get the updated shape
fed_rate_daily_df

,Fed_Fund_Rate_Lower,Fed_Fund_Rate_Lower_Upper,rate_change_bps
Date,,,
2019-01-03,2.25,2.50,25.0
2019-01-04,2.25,2.50,25.0
2019-01-07,2.25,2.50,25.0
2019-01-08,2.25,2.50,25.0
2019-01-09,2.25,2.50,25.0
...,...,...,...
2025-10-16,4.00,4.25,-125.0
2025-10-17,4.00,4.25,-125.0
2025-10-20,4.00,4.25,-125.0


---

## 6. Aggregate the Macro Data <a class="anchor" id="6"></a>

In [433]:
# Combine macro data
macro_data = pd.concat([cpi_daily_df, ppi_daily_df, pce_daily_df, u_rate_daily_df, gdp_daily_df, fed_rate_daily_df], axis=1)

# Check the head
macro_data.head()

,CPI,CPI_yoy_change,PPI,PPI_yoy_change,PCE,PCE_yoy_change,Unemployment Rate,Unemployment_mom_change,Unemployment_yoy_change,GDP,GDP_qoq_change,GDP_yoy_change,Fed_Fund_Rate_Lower,Fed_Fund_Rate_Lower_Upper,rate_change_bps
Date,,,,,,,,,,,,,,,
2019-01-03,252.594,2.147329,202.3,3.266973,102.63,1.941892,3.8,0.000000,-9.523810,20798.73,1.05835,5.61701,2.25,2.5,25.0
2019-01-04,252.594,2.147329,202.3,3.266973,102.63,1.941892,3.9,2.631579,-4.878049,20798.73,1.05835,5.61701,2.25,2.5,25.0
2019-01-07,252.594,2.147329,202.3,3.266973,102.63,1.941892,3.9,2.631579,-4.878049,20798.73,1.05835,5.61701,2.25,2.5,25.0
2019-01-08,252.594,2.147329,202.3,3.266973,102.63,1.941892,3.9,2.631579,-4.878049,20798.73,1.05835,5.61701,2.25,2.5,25.0
2019-01-09,252.594,2.147329,202.3,3.266973,102.63,1.941892,3.9,2.631579,-4.878049,20798.73,1.05835,5.61701,2.25,2.5,25.0


In [435]:
# Check shape
macro_data.shape

(1702, 15)

In [437]:
# Check missing values
macro_data.isna().sum()

CPI                          0
CPI_yoy_change               0
PPI                          0
PPI_yoy_change               0
PCE                          0
PCE_yoy_change               0
Unemployment Rate            0
Unemployment_mom_change      0
Unemployment_yoy_change      0
GDP                          0
GDP_qoq_change               0
GDP_yoy_change               0
Fed_Fund_Rate_Lower          0
Fed_Fund_Rate_Lower_Upper    0
rate_change_bps              0
dtype: int64

In [442]:
# Export the data
macro_data.to_csv("../data/macro_data.csv")